# Chapter 2

**System call**: interface (callable function) to the services provided by a OS. (Ex: write data to a file (by first opening file with system call, then writing to it with another system call)

**Run-tim enviornment (RTE)**: All the software needed for an application to execute using a programming language.

**System call interface**: the link (API) to system calls provided by the OS.

General method to pass parameters into a function is to store each parameter inside a register and pass in the registers. Sometimes, there may be more parameters than registers available (in the case of linux its capped at 5), in that case we store the parameters inside a block or table and the address of the block is passed in as a parameter. We can also push and pop parameters off of a stack. See figure 2.7 for a visual diagram:

![Figure 2.7](./images/f2-7.png)

General linker and loader process:

1. Source files compiled into object files. These object files can then be loaded into physical memory.
2. Linker combines these object files into a single executable file along with the files of used imports/libraries.
3. Loader loads the executable file into memory, and it waits to run on a CPU core.

See figure 2.11 for a visual diagram:

![Figure 2.11](./images/f2-11.png)

**Executable and Linkable Format**: UNIX and Linux system bject files and executable files that include the compiled machine code and a symbol table containing metadata about functions and variables that are referenced in the program.

    main.o - ELF relocatable file
    main - ELF executable

**Monolithic Structure**: No strucutre to organizing functionality of kernel into OS. Everything is inside a single static binary file that runs in a single address space. Ex: UNIX contains kernel and system programs. 

**zombie**: "a process that has terminated, but its parent has not (yet) read its exit status (using the wait() function)"

**orphan**: when a process' parent terminates while it is still running.

# Chapter 3

**Process**: A program being executed by the computer that has access to certain resources depending on where its located, like in the user space or the kernel. The CPU of a computer can execute multiple processes.

The process has a memory layout divided into the following sections: 

**text section** - the executable code

**data section** - global variables

**heap section** - memory that is dynamically allocated during program runtime

**stack section** - stack for holding temporary data storage. examples include local variables, function parameters, and return addresses

The text and data section are fixed sizes as they are set constant for the execution of the program, however the heap and stack change size based on the programs execution.

*Note: a process is not an executable file, but an executable file being executed with instructions loaded into memory being executed one by one and resources allocated.\*

An executing process has the following changing states:

New - The process is being created.

Running - Instructions are being executed.

Waiting - The process is waiting for some event to occur (such as an I/O completion or reception of a signal).

Ready - The process is waiting t obe assigned to a processor.

Terminated - The process has finished execution.

![Figure 3.2](./images/f3-2.png)

**Process control block (PCB)**: Known as a kernel data structure in memory. Its what represents a specific process in the OS. Contents:

 - Process state: New, ready, running, waiting, halted, etc..
 - Program Counter: Address of the next instruction this process will execute.
 - CPU registers: The standard CPU register functions including general registers, index registers, stack pointers, accumulators, etc..\
 - CPU scheduling information: process priority with pointers to sceduling queues and other scheduling parameters.
 - Memory managment information: the value of the base and limit registers and the page tables or segment tables
 - Accounting information: amount of CPU and real time use, time limits, account numbers, process numbers (PID)
 - I/O status information: List of I/O devices allocated to the process (list of open files, etc..)

**Process scheduler**: A feature of the CPU that maintains a data structure of PCB's (eg. doubly linked list). Its goal is to maximize the usage of a CPU to have it running some process at all times. Since a CPU can only run one process at a time, the scheduler handles switching between multiple processes to simulate it running multiple things at once for the user.

Figure 3.4 represents a **wait queue** (processes waiting for a certain event, like waiting for a user input) and a **ready queue** (processes initially created ready for execution) for the process scheduler:

![Figure 3.4](./images/f3-4.png)

**Degree of multiprogramming**: The number of processes currently in memory.

**I/O bound process**: Process that spend more of its time doing I/O (input/output) than actualy computations. Ex: prompt user for numbers to sum up (takes longer to wait on users input than to sum up all numbers).

**CPU-bound process**: Opposite of a I/O bound process in that more of its time is spent doing computations.

Figure 3.5 is a **queueing diagram** which represents a common architecture of process scheduling. The circles represent resources for the processes and the arrows represent the flow of the processes in the system:

![Figure 3.5](./images/f3-5.png)

**CPU scheduler**: Selects a process from the ready queue to allocate a CPU core to. It executes at least once every 100ms and often more frequently, one example for frequent calling is it forcibly removing a process from a core and assigning another process to the core while the first process is waiting for a I/O request. This is known as **swapping**.

**Context switch**: The process of switching from one process being executed on a core to switching to another process being executed on this core instead (this can also be switching from a process to a kernal routine due to a interrupt). This is done in the following steps:

1. A **state save** is performed where we save the current context of a process into its PCB. The info saved includes the value of the CPU registers, the process state, and memory-managment information.
2. A **state restore** is then performed where we load the context of the replacing process (from its PCB) into the CPU core.

Figure 3.6 shows a diagram of a context switch:

![Figure 3.6](./images/f3-6.png)

\*Note: Context switching has a high overhead (overhead meaning a nececary constant cost that isnt dependent upon whatever process or functionality we are running). This is overhead because the system cannot do any work while switching and the switching speed is dependent upon the computers hardware (eg, how many registers its copying over, existince of special instructions, etc..).*

In the case of a system having many registers available and faced with a case of less process than registers, it can simply store pointers to registers as a state save then switch to those registers. But if there are more process than registers then the system resorts to copying register data to and from memory.

A process creating a child process has two scenerios:

1. The parent continues to execute concurrently with its children.
2. The parent waits until some or all of its children have terminated.

There are also two address-space possibilities for the new process:

1. The child process is a duplicate of the parent process (it has the same
program and data as the parent).
2. The child process has a new program loaded into it.
   (Both of these follow the fork() and fork() then execve() process respectively)

When a parent process creates a child process (using fork()) if it utilizes a wait() system call it then moves itself off of the ready queue until the termination of the child.

Figure 3.9 demonstrates this:

![Figure 3.9](./images/f3-9.png)


**Cascading termination**: If a parent process is being terminated than all its child process are terminated also. This feature is dependent on the OS, some systems create orphan processes instead.

The UNIX system addresses orphan processes by assigning the init process (the root process) as the new parent to the orphan process. 

Processes executing at the same time (concurrently) can either be independent processes or cooperating processes.

**Independent process**: This process does not share its data with any other process currently executing in the system.

**Cooperating process**: This process can be affected or can affect other processes currently executing in the system.

**Interprocess communication (IPC)**: A mechanism that allows for processes to send and receive data from one another. This mechanism follows two models **shared memory** where a portion of memory is allocated to be shared among processes that they all read and write to, and **message passing** where a message pathway is established among processes that holds messages in queue form.

Figure 3.11 demonstrates both forms of IPC:

![Figure 3.11](./images/f3-11.png)



Message passing is slower (due to them being built using system calls that occupy the kernel) but simpler (since no conflicts need to be avoided) and is thus useful for small amounts of data. Is also used for distributed systems where communication is done by multiple computers (with their own memory) connected by a network.

Shared memory is more expensive and complicated but is faster since we can just implement routine memory reading for each process as the IPC.

A method in which memory sharing works is using two types of buffers: "The **unbounded buffer** places no practical limit on the size of the buffer. The consumer may have to wait for new items, but the producer can always produce new items. The **bounded buffer** assumes a fixed buffer size. In this case, the consumer must wait if the buffer is empty, and the producer must wait if the buffer is full."

In this case the producer has a pointer to the last written point in the buffer and if it isnt full it writes to it (otherwise it waits for it to be empty. The consumer has a pointer to the last read point in the buffer, if the read pointer is equal to the write pointer it waits otherwise it reads from the buffer then increments the read pointer.

In message passing we must establish a **communcation link** for processes to send and receive messages from, There are different implementation methods for the link:

- **Direct communication (aka symmetry)**: Each process must reference which processes they are sending or reading info from (each processes pair has exactly one link):

      send(P, message), receive(Q, message)
  
- **Asymmetry**: Only the message sender names the recipent, while the receiver doesnt have to name the sender
  
      send(P, message) — Send a message to process P.
      receive(id, message) — Receiveamessagefromanyprocess.
  
- **Indirect communication**: messages are sent to and received from mailboxes, or ports. Processes can write to one or more mailboxes and read from one or more mailboxes

    send(A, message) — Send a message to mailbox A.
  
    receive(A, message) — Receive a message from mailbox A.


**Synchronization**: Message passing methods for communication between processes using send() and receive() functions:

- **Blocking send**: The sending process is blocked until the message is received by the receiving process or by the mailbox.
- **Nonblocking send**: The sending process sends the message and resumes operation.
- **Blocking receive**: The receiver blocks until a message is available.
- **Nonblocking receive**: The receiver retrieves either a valid message or a null.

**Buffering**: A queue that holds messages being exchanged between two processes. The following possible buffering techniques are possible:

- **Zero capacity**: The queue has a maximum length of zero; thus, the link cannot have any messages waiting in it. In this case, the sender must block until the recipient receives the message.
- **Bounded capacity**: The queue has finite length n; thus, at most n messages can reside in it. If the queue is not full when a new message is sent, the message is placed in the queue (either the message is copied or a pointer to the message is kept), and the sender can continue execution without waiting. The link’s capacity is finite, however. If the link is full, the sender must block until space is available in the queue. 
- **Unbounded apacity**: The queue’s length is potentially infinite; thus, any number of messages can wait in it. The sender never blocks.

**Pipe**: One of the first IPC mechanisms in early UNIX systems. They act as a channel of communication for two processes. Two common types of pipes are ordinary pipes and named pipes.

**Ordinary pipe**: Standard unidirectional communication system. The producer process writes to one end of the pipe known as the **write end** and the consumer process reads from the other side of the pipe known as the **read end**. If the pipes need two-way communication (they both need to read and write to each other) then we must open up two pipes.

The figure 3.20 below demonstrates a pipe process 
(note that fd[0] is the read end of the pipe, and fd[1] is the write end 

and in this case the read end of the parent (fd_p[1]) is closed as its only writing and the write end of the child is closed (fd_c[0]) as its not writing)

![Figure 3.20](./images/f3-20.png)

**Named pipe**: Also a unidirectional pipe (half-duplex transmission only). But now the pipe remains until its specifically deleted from the file system (acts as a file) and can be used by multiple processes (ordinary pipe deletes itself after its communicating processes have terminated). Follows a FIFO data structure with the possibilty of multiple processes as writers and multiple processes as readers. "Additionally, the communicating processes must reside on the same machine. If intermachine communication is required, sockets must be used."

**Socket**: An endpoint for communication, typically using some network service like SSH, FTP, and HTTP along with a port. These are used as a communication tool for a pair of processes communicating over a network (one socket is assigned to each process). Example connection:

1. Client process host X with IP address 146.86.5.20 initiates a request for a connection with a web server 
2. This web server is listening for connections on port 80
3. The web server reads the clients request and returns a port number for it (greater than 1024) that turns out to be 1625

Now the host and web server are able to exchange information specifically with each other by specifying each others ports as the destination. (Host sends data to port 80 and servers sends data to port 1625). This means that connections containing ports are unique, so if another process wanted to communicate with the web server it would need its own port value assigned (something greater than 1024 and not equal to 1625).

This is demonstrated visually in figure 3.26 below:

![Figure 3.26](./images/f3-26.png)

**Remote Procedure Calls (RPC)**: A abstraction/wrapper for using the procedure-call mechanism across systems with network connections. Uses a message system like IPC where we send information to a listening port. This information includes the function to execute and the parameters to pass to that function. The response on the function is then sent back as a message to the caller computer over the network.

**Stub**: known as a protocol compiler, it handles all the details of communicating on the reseiving and sending end of both the client side and server side. Handles the following process:

- creates a message buffer
- packs the function identifier and arguments into the buffer (aka message serialization or
**marshalling** of arguments)
- sends the message across the network to the destination RPC server (handled by the run-time
library)
- waits for the reply
- unpacks the return code and/or results (aka message deserialization or unmarshalling)
- returns to the caller

**Marshalling**: parameter marshalling handles the issue of the client side and server side having different data representations of the data being shared (can be different in big-endian or little-endian systems). One method is having code that converts to a shared **external data representation (XDR)** that both send to each other and then know how to convert back to their specific data format to read.

# Chapter 4

**Thread**: A single unit of CPU utilization. It comprises of a thread ID, program counter (PC), a register set, and a stack. It shares its code and data section along with system resources (like open files and signals) with other threads. Usually a process has one thread of control and if it has multiple then it can do multiple computations at once this difference is **single-threaded** and **multithreaded**. Figure 4.1 shows the difference between the two processes:

![Figure 4.1](./images/f4-1.png)

**Thread control block (TCB)**: Data strucutre that contains thread specific information thats needed to manage the threads (thread ids, register sets, stacks, etc..). Also contains scheduling info (using priority) and a pointer to the process control block (PCB). Also contains **Thread-Local Storage (TLS)** which stores data globally to be shared across threads. 

One example of a multithreaded system is a web server, the server will create a separate thread that listens for client requests. When a request is made, the server creates a new thread to service the request and resumes listening for additional requests. This is illustrated in Figure 4.2.

![Figure 4.2](./images/f4-2.png)

4 major categories of benefits of multithreaded programming

1. **Responsiveness**: Allows the user to interact with an application even while some of the application is blocked or performing a length operation. (Ex: Click button to do length calculation and can still use website for other features while its calculating).
   
2. **Resource sharing**: Several threads of activity in the same address space sharing the same data comparing to multiple processes needing to explicitly establish shared memory or message pipes.

3. **Economy**: It is computational cheaper to create multiple threads that work on the data in the process they belong to and then to perform context switches among the processes than to allocate memory and resources for process creation.

4. **Scalability**: Threads can run in parallel on multiple processing cores. Whereas a single processor can only run one one processor even if we have more available.

Below is a figure 4.2S of how a single vs multithreaded process differ:

![Figure 4.2S](./images/f4-2S.png)

**Multicore system**: "multiple computing cores on a single processing chip where each core appears as a separate CPU to the operating system".

**Concurrency**: Supporting more than one task by allowing all the tasks to make progress

**Parallelism**: Can execute more than one task at once

**Concurrency on a single computing core**: If we were to try and implement concurrency (running different code at the same time) on a single computing core we would have to simply just interleave the computation one after the other to create the illusion of parallel computation. This is because the processing core is capable of executing only one thread at a time. See figure 4.3 for a diagram of this:

![Figure 4.3](./images/f4-3.png)

This is an example of how it is possible to have concurrency without parallelism.

**Concurrency on a multicore system**: Can execute a thread on each core. See figure 4.4 for a diagram of this:

![Figure 4.4](./images/f4-4.png)

These are the 5 main challenges in programming for multicore systems:

1. **Identifying tasks**: Evaluating applications and finding areas where tasks in the code are independent of each other and can be split into concurrent tasks.
2. **Balance**: When splitting a task into seperate parallel tasks, make sure that it can be split evenly into equal contributing tasks.(Make sure one core isnt doing way more work than the rest).
3. **Data splitting**: Ensure that the data is split evenly among each of the threads/cores
4. **Data dependency**: Making sure that the data being split for each parallel task isnt dependent upon another part of the data being used for a parallel task. If it is then we need to change the execution of the tasks to be synchronized.
5. **Testing and Debugging**: A task being run in seperate parallel tasks is much harder to test and debug since there are many different execution paths possible. (Ex: scheduler can run threads in any order and switch threads at any time (context switches)).

**Amdahl’s Law**: Formula for potenial performance gain for an application from adding extra computing cores based on the percentage of serial (nonparallel) and parallel components. S is a percentage value for the amount of application execution that must be performed serially and N is the amount of processing cores.

    speedup <= [1 / (S + ((1-S)/N))]

    ex: application is 75% parrallel and 25% serial, if we use two processing cores we get: [1 / (0.25 + ((1-0.25)/2))] = 1.6 (1.6x in speedup)

    lim N->inf [1 / (S + ((1-S)/N))] = 1/S (meaning that no matter how many cores we add the performance increase eventually caps at 1/S)

See figure 4.4S for a graphing of the limit approach:

![Figure 4.4S](./images/f4-4S.png)

**Data parallelism**: Distributing subsets of the same data among threads that perform the same task. Ex: To sum the data from 1 to n can split up the data in half (1 to n/2 and n/2 +1 to n). Then send each portion a thread that sums the data, then sum the results.

**Task parallelism**: Each thread performs a unique task on either a unique portion of the data or the entire dataset itself. 

Figure 4.5 shows the difference between these two (a hybrid version of combining both parallelisms is possible):

![Figure 4.5](./images/f4-5.png)

**User threads**: Threads handled within a process, the kernel has no idea or support for the threads. The OS only schedules the process containing the threads. Managed by a thread-library inside the process, this gives us more control over the threads. For example, we can implement our own scheduling policy for the threads along with implementing preemptive features for running threads. We can also schedule our threads non preemptively and only switch on a yield. They are also much faster than kernel calls as they use library level functions instead of system functions. Some disadvantages are since the threads are hidden from the OS, our process scheduling system treats the process the same as all other processes (regardless if they have multiple or no threads). It can run a process with idle threads or if a thread makes a system blocking call (like I/O request) the entire process is blocked. We can implement communication feature between the kernel thread manager and the user-level thread manager.

**Kernel threads**: Threads directly supported by the kernel. Meaning the kernel manages the scheduling of the kernels which can replace process scheduling and provide a benefit of faster switching (context switches faster than process switches) along with sharing of data with same address space and only having to deal with each owns program counter, stack, etc..

3 common methods of relationships between user threads and kernel threads (and one extra one):

1. **Many-to-One Model**: Many user level to one kernel thread, user space thread library manages threads so simple. But, entire system is blocked if one thread blocks and all bottleneck into one kernel thread so removed benefit of parallelism. See figure 4.7

![Figure 4.7](./images/f4-7.png)

2. **One-to-One Model**: Each user thread creates a kernel thread, this allows for parallelization and allows for other threads to run when one thread blocks. Only drawback is that it can be computationally expensive to create a kernel thread for each user thread if a lot of threads are required. Linux uses this one.

![Figure 4.8](./images/f4-8.png)

3. **Many-to-Many Model**: Many user threads are mapped to a equal or smaller number of kernel threads (the number of kernel threads is specific upon the application or system (like cores). This allows for paralization of thread to thread but also can limit the amount of kernel threads by mapping many user threads to one kernel thread or only a couple for example.

![Figure 4.9](./images/f4-9.png)

4. **Two-level Model**: Same as the Many-to-Many model but also allows for a one-to-one mapping for threads as well. Meaning you can have both.

![Figure 4.10](./images/f4-10.png)

Due to modern systems having many cores the issue of limiting kernel threads isnt too much of an issue anymore so most systems use the One-to-One model.

**Asynchronous threading**: Parent creates child thread then parent resumes its execution so the two can run concurrently and independently of each other. (Usually little data sharing since they are independent).

**Synchronous threading**: Parent creates one or more children threads and waits for them all to complete before it continues its execution. Usually has lots of data sharing and the parent combines the info from all the children threads somehow.

**Pthreads**: POSIX standard (IEEE 1003.1c) API for thread creation and synchronization. See figure 4.11 of example code:

This figure has a thread execute the runner function that modifies the global sum variable. The pthreaed_join is a wait for the main thread waiting for the child thread it created to finish. Then it prints the modified sum value.

![Figure 4.11](./images/f4-11.png)

Can run multiple child threads at once to all contribute to the sum at once. See figure 4.12:

![Figure 4.12](./images/f4-12.png)

**Implicit threading**: Move the process of creating and managing threads from application developers to compilers and run-time libraries (Simplifies the process of using threads for developers, now they just need to specify the tasks that can be used in parallel).

**Explicit threading**: Opposite of implicit threading, the developer handles the creation and managment of threading. This is done by either creating a library entirely in the user space or by creating a kernel level library supported by the OS. (PThreads can be used as either a user level or kernel level library). Things the developer handles e.g. explicitly synchronize threads, exchanging messages, make transactional operations etc..

**Thread pool**: At the start up of a program that uses threads create a number of threads and place them into a pool waiting for work. Now whenever a request is submitted we check if a thread exists in the pool, if it does then we service the request and wait for more requests. If a request is made and no threads are currently available then the task is added into a waiting queue where it waits for an available thread. Once a thread completes it returns to the pool ready to take on another task. This is demonstrated in Figure 4.12S:

![Figure 4.12S](./images/f4-12S.png)

This provides the benefit of not having to wait for a thread to be created at each service/task request, having a limited number of threads to avoid overcreating threads to handle an infinite task (ex: website gets spammed with requests knowing that each request creates a thread to handle it), and seperating the task scheduling from task creation in that we can add custom features for scheduling tasks like adding delays or periodic runs.

**Fork Join**: Thread creation strategy in Figure 4.11. A parent process creates (forks) one or more child threads and waits for them all to terminate and join their results together. Is a synchronos model as it must wait for the results of the threads to get its final result. Usually implemented using explicit thread creation but can be done using implicit thread creation. In the implicit version we specify parallel tasks to perform instead of explicity declaring and creating the threads ourselves. "It can be thought of as the synchronous version of thread pools in which a library determines the actual number of threads to create". The overal fork join is demonstrated in Figure 4.16:

![Figure 4.16](./images/f4-16.png)

**OpenMP**: API for C/C++ programs that identifies parallel regions as blocks of code that can run in parallel. Creates as many threads as processing cores in the system, then executes them all in parallel.

When you call fork in a program that is using threads, the fork only duplicates all threads if there is no exec() function to take over the fork. If there is then we only duplicate the calling thread of fork() since we know that exec() will replace whatever code is in the fork currently.

**Signal**: signal used in UNIX to alert a process that a specific event has occured (event specified by the type of alert). There are two types of signals:

- **synchronous signal**: Signal is delivered to the process that ran some code that caused the signal
- **asynchronous signal**: Signal is delivered to a process from an external source. (Ex: Control+C or a process timer completing)

**Signal handling in Multithreaded programs**: 4 methods exist for handling a signal when running threads:

1. Deliver the signal to the thread to which the signal applies.
2. Deliver the signal to every thread in the process.
3. Deliver the signal to certain threads in the process.
4. Assign a specific thread to receive all signals for the process.

A synchronous signal is always sent to the thread that generated it.

A asynchronous signal is sent to all threads in the case of terminating a process for example (Control+C)

An asynchronous signal can also be sent to all threads that are not blocking it, but usually only sent to the first thread found that is not blocking it as we only need to handle a signal once.

Can execute a signal in POSIX using the following function: pthread kill(pthread t tid, int signal)  (tid is the thread id to send the signal to).

**Thread cancellation**: Terminating a thread before it has completed. A thread that is to be cancelled is refered to as a **target thread**. Which can be cancelled with 2 possible methods:

- **Asynchronous cancellation**: A thread immideatly terminates the target thread
- **Deferred cancellation**: A thread periodically checks if it has become a target thread, thus each thread can terminate itself. A thread checks if it is to be cancelled when it reaches a checkpoint that is defined as a safe location to cancel. We need a safe location as we run the risk of cancelling a thread that is updating some shared data used by other threads. These are known as **cancellation points**. Ex: the read() system call is a cancellation point that allows cancelling a thread that is blocked while awaiting input from read(). We can also manually add cancellation points in our code by adding the method pthread_testcancel(). This method wont return and will cancel the thread if a cancellation request is on, otherwise it returns and the thread code proceeds.

/* cancel the thread */
pthread_cancel(tid);
Then pthread_join() returns when a thread is cancelled.

A thread can define its type and if it is enabled to be cancelled or not (the cancel state can change throughout the threads execution pthread_setcanceltype()). See Figure F4.16T:

![Figure 4.16T](./images/f4-16T.png)
 
**Cleanup handler**: Function that is called when a thread is cancelled. Takes any information that the thread is working on and releases it before the thread is cancelled.

# Chapter 5

**Process Cycle**: The cycle a process being executed goes through. The process begins with a CPU burst and then is followed by a I/O burst. This cycle can loop over and over until it ends with a CPU burst using a system request to terminate execution.

**CPU burst**: The CPU core running some code where code is being executed that does some sort of computations (like addition or calculation of some kind). There can be short CPU bursts and long CPU bursts. Short bursts have a high frequency of small calculations and a long burst has a few long computations. An example of a short burst program is a I/O bound program as it interrupts a lot to take in input or output.

**I/O burst**: A program pausing a lot to handling either outputting data or taking input data from the user.

**CPU Scheduler**: Scheduling algorithms that handles a queue of processes wanting to be executed (represented as PCBs). When a CPU core is idle the scheduler selects a process from the queue to occupy that core with. A queue can be a FIFO queue, priority queue, tree, or an unordered linked list. Scheduling decisions are made under the following 4 scenerios:

1. A process switches from the running state to the waiting state (ex: I/O request or wait() for termination of child process)
2. A process switches from the running state to the ready state (ex: interrupt occurs)
3. A process switches from the waiting state to the ready state (ex: completion of I/O)
4. A process terminates

For situations 1 and 4 we simply select the next process in the ready queue. This is known as a **nonpreemptive** or **cooperative** scheduling scheme.
For situations 2 and 3 we have a **preemptive** scheduling scheme in that we have a choice on what process to select next. 

**Dispatcher**: A component from the CPU scheduler that is responsible for giving control of a CPU core to a process that was selected by the scheduler. It handles the following:

- Switching context from one process to another
- Switching to user mode
- Jumping to the proper location in the user program to resume that program

**Dispatch latency**: The time taken by the dispatcher to stop a process and start the running for another process.
Figure 5.3 demonstrates this:

![Figure 5.3](./images/f5-3.png)

The following definitions below is the criteria used for selecting a scheduling algorithm (based on our specific circumstances)

**CPU Utilization**: The % of time that the CPU is occupied with work. In a real system is should range from 40-90 % occupied.

**Throughput**: The amount of processes that are being completed per time unit. This can be a measure of how much work a CPU is getting done. Long processes can have a throughput of one process over several seconds, short processes can be 10's of processes per second.

**Turnaround time**: The amount of time taken for the process to run, starting from initialization to termination. It includes the time spent waiting in the ready queue, execution time on CPU, and waiting and performing I/O.

**Waiting time**: The amount of time a process is in the waiting queue. (Since scheduling algs have no effect on the time it takes for a process to handle I/O this can be useful for evaluating scheduling algs).

**Response time**: The time taken from when a request is produced to when the first response is produced. (The time it takes from when a user presses enter to when the program outputs some information to them). "The time between when process enters the ready queue and it finishes execution on CPU".

A goal of a scheduling algorithm is to maximize CPU utilization and throughput and to minimize the remaining timeing criteria.

In most cases optimize the collected average of all criteria but we can also maximize or minimize the values of specific criteria.

**First-Come-First-Served scheduling**: The process that requests to use the CPU first gets selected first. This is done with a FIFO queue where new processes get added to the tail of the queue and the processes at the head of the queue get selected first. One of the simplest scheduling algs but also usually has one of the longest average waiting times. As the algorithim is nonpreemptive, a process keeps a CPU until it is either terminated or requests I/O. Consider the following examples:

![Figure 5.S](./images/f5-s.png)

**Shortest-Job-First scheduling**: When a CPU is available this scheduler selects the process that has the shortest next CPU burst. This scheduling algorithm is provably optimal in providing the minimum average waiting time for a given set of processes.
Ex: Process - Burst Time

    P1 - 6
    P2 - 8
    P3 - 7
    P4 - 3

    average waiting time is (p1 + p2 + p3 + p4) -> (3 + 16 + 9 + 0)/4 = 7 milliseconds (For FCFS the average waiting time would be 10.25 ms)

![Figure 5.1S](./images/f5-1s.png)

Issue with the algorithm is that there is no way to know what the next CPU bursts are going to be for the processes. So we can predict the next CPU burst as a exponential average of the measured lengths of previous bursts. 

Let tn be the length of the nth CPU burst, and let gn+1 be our predicted value for the next CPU burst. Then, for α (between 0 and 1) we have:

    gn+1 = α*tn + (1-α)gn

Non preemptive SJF: When selecting a job from the waiting queue it selects the process with the shortest next CPU burst. But while a CPU is running a process it must wait for it to finish or interrupt.

Preemptive SJF: Even while running a process its contantly checking for jobs being added to the waiting queue and if it finds one thats shorter than the one currently being executed it pauses the current process and switches to the shorter one.

**Shortest Remaining Time First (SRTF)**: SJF with a preemptive selection. Long running CPU bound jobs can **starve (aka indefinite blocking)** (if new short jobs keep arriving).

**Round-Robin scheduling**: Uses the same queue and selection logic as First-Come-First-Served with a FIFO queue. However, a small chunk of time called a **time quantum** is defined (usually between 10 to 100 miliseconds). This time quantum is used as the maximum allocation time a process can use a CPU before its interrupted and pushed to the end/tail of the waiting queue. The next process is then selected from the head and the process repeats. This results in the waiting queue being treated as a circular queue (or round robin). Given the following example Process - Burst time:

        P1 - 24
        P2 - 3
        P3 - 3

With a time quantum of 4 miliseconds we get the following execution:

![Figure 5.33](./images/f5-33.png)

P2 and P3 finish before the time quantum thus only execute once and the large process p1 gets split up into 4 milisecond chunks and finishes at the end. P1 waits for 6 milliseconds (10 − 4), P2 waits for 4 milliseconds, and P3 waits for 7 milliseconds. Thus, the average waiting time is 17/3 = 5.66 milliseconds.

"If there are n processes in the ready queue and the time quantum is q, then each process gets 1/n of the CPU time in chunks of at most q time units. Each process must wait no longer than (n − 1) × q time units until its next time quantum."

Goal for time quantum is to try and make it bigger than 80% of the process CPU bursts as if its too big then we just have a regular FCFS alg, if its too small then we add in a lot of extra overhead computation of the context switching.

round robin is **fair** meaning each job gets an equal shot at the CPU (doesnt matter if one process is much longer than the rest). 
A disadvantage of the algorithm is that it can be extra long (longer than FCFS alg) if all the processes have an equal CPU burst time (then were just adding overhead switch computations for no reason/benefit).

**Priority Scheduling**: A priority value is assigned to each process in the waiting queue. (The SJF algorithm is a special case of the general priority-scheduling algorithm).

**Aging**: Technique to solve starvation in priority queues. Iteratively increase the priority of all processes in the waiting queue, that way even if a process is added with a low priority and new processes are constantly added with high priorities it will eventually increase the low priorities priority to a high enough value to get selected.

To combat starvation could also combine priority scheduling and round robin scheduling where processes with the highest priority are still selected first, but if there are multiple processes at the same priority value we use round robin for those processes.

**Multilevel queue**: A distinct queue for each priority, this removes the nececity of performing O(n) search everytime we want to find the process with the highest priority value. This can be combined with executing round robin on the queues that have more than one process (processes with the same priority value). See figure 5.7 for a diagram:

![Figure 5.7](./images/f5-7.png)

We can also seperate processes of different types at different priorities. These different type processes can specify different queues and specific scheduling algorithms. For example seperating foreground (interactive) and background (batch) processes. Foreground could required a round robin scheduler and background could use a first come first serve scheduler. See figure 5.8:

![Figure 5.8](./images/f5-8.png)

This example above is preemptive, meaning no process from the queue can be ran if the queue at a higher level priority is not empty. Also, while a process is running, if a process with a higher priority is added (goes to a higher different queue) then the current process is preempted (dumped) and the higher new process is executed.

Can also time-slice among the queues and give each queue/priority level a certain portion of CPU time that it has to divide among its processes. We can give 80% of CPU time for round robin scheduling among foreground processes and 20% of CPU time to the background processes queue on FCFS scheduling.

**Multilevel Feedback Queue scheduling algorithm**: A multi level queue that seperates processes by CPU bursts. It also gives the flexibility of processes to move between queue's. If a process uses too much CPU time it will move down a queue. Processes that wait too long in a low priority queue can also be moved up to higher priority queues. (I/O bound processes usually have short CPU bursts and are thus put in the highest priority queue).

Figure 5.9 demonstrates one version of a multilevel feedback queue. If a process takes more than 8 ms to complete it is moved down to the second priority queue, if it then takes more than 16ms to complete it is moved to the bottom queue which is then FCFS instead of round robin (RR). These are also preemptive meaning we can drop curring running processes at any time for quicker processes. Also to prevent starvation the processes in the bottom queue are gradually moved up to higher queues over time.

![Figure 5.8](./images/f5-8.png)

MFQ is one of the most complex algorithms but is the most general CPU-scheduling algorithm. It can be defined with the following parameters:

- The number of queues
- The scheduling algorithm for each queue
- The method used to determine when to upgrade a process to a higher-priority queue
- The method used to determine when to demote a process to a lower-priority queue
- The method used to determine which queue a process will enter when that process needs service

**Load-sharing**: Multiple CPU cores are available, allowing for multiple threads to run in paralel.

**Multiprocessor**: Applies to the following archetictures: Multicore CPUs, Multithreaded cores, NUMA systems, and Heterogeneous multiprocessing.

**Asymmetric multiprocessing**: One core has access to the system data handling scheduling decisions, I/O processing, and all other system processing. Known as the master server while all the other processors only handle the user code (aka the processes being executed). This is a simple method but has the drawback of the master server being a bottleneck.

**Symmetric multiprocessing (SMP)**: Each processor is self-scheduling, meaning each processor reads the queue and selects a thread to run. There are two strategies for organizing the threads to be selected:

1. All threads are in a common ready queue. (Possibility of race condition so must make sure two processors dont select the same thread (could use locking but this makes ready queue a bottleneck)).
2. Each processor has its own private queue of threads. (This requires load balancing algorithms to properly distributed threads among all private queues).

Figure 5.11 shows a diagram of both of these strategies:

![Figure 5.11](./images/f5-11.png)

**Real-time system**: Handles multiple processes at once and makes sure that task completion is completed by a certain deadline.

**Soft real-time system**: The scheduler is best effort meaning it cant guarantee when a real-time process will finish. (performance degrades if deadlines cannot be met (event latency > time budget)).

**Hard real-time system**: The scheduler must guarantee completion by a deadline. If the deadline is not met then the system fails and it counts the process as not executed (even if it completed after the deadline).

**Event latency**: The amount of time from when an event occurs to when that event is serviced. (Ex: Automatic car detects obstrucution and then hits brakes).

**Interrupt latency**: The amount of time from when an interrupt signal arrives at the CPU to when the CPU starts the routine (or function) that services the interrupt. (From interrupt being called to interupt function being executed). In between interrupt being called and the method executing the CPU must first complete its current instruction, determine the interrupt type, save the current state, execute the interrupt function. Figure 5.18 demonstrates this:

![Figure 5.18](./images/f5-18.png)

**Dispatch latency**: The amount of time taken for a scheduling dispatcher to stop a process and start another one. The entire process is demonstrated in figure 5.19

![Figure 5.19](./images/f5-19.png)

"The most effective technique for keeping dispatch latency low is to provide preemptive kernels."

The conflict phase of dispatch has two components:

1. Preemption of any process running in the kernel
2. Release by low-priority processes of resources needed by a high-priority
process

The dispatch phase of dispatch is scheduling the real-time process onto the CPU.

**Periodic task**: A scheduling technique for hard real-time processes. Each process is represented as a periodic process where we have a variable t that defines the processes fixed processing time, a deadline d by which it must be serviced by, and the CPU is requested for a time period of p. 0 <= t <= d <= p. The rate of a periodic task is 1∕p. We can thus use these values to assign priorities according to a process’s deadline or rate requirements. Figure 5.20 demonstrates this:

![Figure 5.20](./images/f5-20.png)

**Admission-control algorithm**: Each process has to announce its deadline to the scheduler, this algorithm decides wether to admit the process and thus guaranteeing that the process will complete on time or to reject a process as it cant guarantee that will be serviced by its deadline. This is a nececary feature in hard-time systems.

**Rate-Monotonic scheduling**: schedules periodic tasks using a static priority policy with preemption. A priority is assigned based on the inverse of a tasks period, meaning the shorter the period the higher the priority. This is a simple algorithm that doesnt require knowledge of the deadline. Consider the example:

    P1: {p1 = 50, t1 = 20, d1 = 50}
    P2: {p2 = 100, t2 = 35, d2 = 100}

    1. Make sure its possible that both processes can complete before their deadline:
        sum_i t_i/p_i <= 1
        t1/p1 = 20/50 = 0.40
        t2/p2 = 35/100 = 0.35
        == 0.75 < 1 therefore we utilize 75% of the CPU, since this is less than 100% we can schedule and complete both
    
    2. Assign p1 before p2 because p1 period of 50 is less than p2 period of 100.

    3. P1 completes at time 20 (due to its t) then P2 starts running and runs until time 50, it is then preempted by anoter instance of P1 being inserted (so pauses with 5 left) (This only happens because we reach the end of P1, another instance of P1 could have been added before that time but the CPU committed p1 time to run the processes it witnessed at time t-50. So once it ends at t=50 it restarts its check and finds another p1 while its executing P2).

    4. The second P1 then completes at time 70 (50 start + 20 execution)
    5. P2 is resumed and finishes its remaining 5 at time 75 (also meeting its deadline of 100).
    6. The CPU is idle finishing up the assigned CPU time for p2 going until 100
    7. The process then repeats over to step 1.

This example is demonstrated in Figure 5.22:

![Figure 5.22](./images/f5-22.png)

**Earliest-deadline-first (EDF) scheduling**: assigns priorities dynamically according to deadline. The earlier the deadline, the higher the priority. With each addition of a new process, priorities on all the waiting processes are adjusted based on the deadline of the newly added process (this differs from RM scheduling in that priorities are assigned statically (just take the smallest existing value)). This algorithm is preemptive, and does not require tasks to be periodic or to know their burst-times (meaning this alg can be easily implemented).Consider the following example that would fail to run under RM (Figure 5.24 provides a visual diagram):

    P1: {p1 = 50, t1=d1 = 25}
    P2: {p2 = 80, t2=d2 = 35} (EDF doesnt have periodic we just have this to show that it would fail for RM)

    1. P1 has highest priority with shortest deadline so it runs first, completes, then p2 is ran
    2. In RM p2 would be preempted by a new p1 at time 50, EDF keeps running P2 because P2 has a higher priority than P1 because its next deadline (at time 80) is earlier than that of P1 (at time 100).
    3. P1 and P2 both meet there deadlines:
    4. Process P1 again begins running at time 60 and completes its second CPU burst at time 85, also meeting its second deadline at time 100. 
    5. P2 begins running at this point, only to be preempted by P1 at the start of its next period at time 100. P2 is preempted because P1 has an earlier deadline (time 150) than P2 (time 160). 
    6. At time 125, P1 completes its CPU burst and P2 resumes execution, finishing at time 145 and meeting its deadline as well. 
    7. The system is idle until time 150, when P1 is scheduled to run once again.

![Figure 5.24](./images/f5-24.png)

EDF theoretically, can schedule processes so that each process can meet its deadline requirements and CPU utilization will be 100 percent. In practice, however, it is impossible to achieve this level of CPU utilization due to the cost of context switching between processes and interrupt handling.

***Not in Textbook (covered in slides)***:

**Lottery scheduling**: every process a number of lottery tickets on each time slice, randomly pick a winning ticket and run the winning process. 

How to assign tickets? to avoid starvation, every job gets at least one ticket

1. give most tickets to short running processes, and fewer to long running processes
(approximating SJF scheduling)
2. give most tickets to high-priority processes, and fewer to low-priority processes (approximating
priority scheduling)

For priority inversion: donate tickets to the process you are waiting on

**Least-Laxity-First (LLF) scheduling**: "dynamic priority scheduling algorithm. It means that every instant is a scheduling event because laxity of each task changes on every instant of time. A task which has least laxity at an instant, it will have higher priority than others at this instant." Goal: minimize the laxity (the amount of time a task can be delayed without missing its deadline) of each task. **slack time** = time until deadline - remaining execution time.

![Figure 5.25S](./images/f5-25S.png)

More complicated as many more computations needed and it updates priorities dynamically but, it can detect missed deadlines early and do not execute those jobs at all. (give a job 10% of tickets is equivalent to giving it 10% of CPU cycles on average). 

# Chapter 6

**Cooperating process**: A process that can affect or be affected by other processes. This means that the process can share addresses of code and data either through shared memory or message passing.

**Race condition**: Multiple processes acess and manipulate the same data at the same time. This results in the outcome of the value being dependent on the order in which the access to the variable takes place. 

For example, consider a producer and a consumer thread running concurrently:

The producer has a job to add data and increment the count variable
The consumer has a job to use data and decrement the count variable

In this scenerio we want the producer to add data to an existing count of 5, meaning we have 6 blocks of data, then we want the consumer to use a block of data and end with a total of 5 blocks of data. Here is a concurrent scenerio:

    T0: producer  register1 = count           {register1 = 5}
    T1: producer  register1 = register1 + 1   {register1 = 6}
    T2: consumer  register2 = count           {register2 = 5}
    T3: consumer  register2 = register2 − 1   {register2 = 4}
    T4: producer  count = register1           {count = 6}
    T5: consumer  count = register2           {count = 4}

**Critical section**: A section of code where a process is updating some data that is shared between other processes. This lets us define the "critical-section problem" where we have to make sure that no more than one process is executing inside a critical section at once. This forces processes to synchronize their execution around critical sections which results in processes copareativly sharing data. A code block can thus be demonstrated in terms of a **entry section**, **critical section**, **exit section**, and **remainder section**. A process has to request permission to enter a critical section when it reaches an entry section. This section strucutre is demonstrated in Figure 6.1

![Figure 6.1](./images/f6-1.png)

3 criteria for a satisfactory critical section problem:

1. **Mutual exclusion**: If a process/thread is executing in a critical section then no other thread can be executing its critical section
2. **Progress (liveness)**: Only processes that are not in their remainder section can decide if another process can execute its critical section and they cannot postpone allowing another process to execute its critical section indefinetly.
3. **Bounded waiting**: Bounded number on the amount of times processes can enter their critical sections after another process has requested to execute its critical section.

**Peterson’s solution**: Software approach that makes sure two processes dont run their critical section at the same time. The algorithm is demonstrated with two processes process i (Pi) and process j (Pj). Each process shares the flags array and the turn array as shared data. Then each algorithm gives permission for the other process to run its critical section and waits for it complete before it jumps into its critical section. Essentially, each process sets the turn to the other process and whichever process executes the turn assignment last is the one that waits. Figure 6.3 demonstrates this algorithm:

![Figure 6.3](./images/f6-3.png)

(Ex: Pi sets turn=j, then Pj sets turn=i so Pj waits for Pi to compelete. Then Pi completes and sets its flag[i] = false allowing Pj to run its ciritical section)

Petersons validity: 

    mutual exclusion - turn can either be 0 or 1 meaning one process has to wait for the other
    progress - process prevented from exiting remainder section with wait while loop waiting for permission to run its critical section
    bounded - other process guaranteed to enter its critical section after waiting for at most one critical section due to each process setting its flag to false in its remainder section.

**Memory model**: abstract representation of how memory is managed by the OS. How a computer determines what memory rules and guarantees it will provide to an application. Memory model has 2 categories:

1. **Strongly ordered**: where a memory modification on one processor is immediately visible to all other processors.
2. **Weakly ordered**: where modifications to memory on one processor may not be immediately visible to other processors.

**Memory barriers/fences**: Forces read and writes of specified variables to execute in the order specified. This is important in multithread executions as the compiler rearanges lines of code that arent dependent upon each other for efficiency. (Very low level method and usually only used by kernel developers). Ex:

    x = 100;
    memory_barrier(); 
    flag = true;
    
    This ensures that the assignment to x happens before the assignment to flag

**Atomic operation**: Nothing can interrupt (or context switch) when an atomic operation is executing. 

Figure 6.5 & 6.6 demonstrate how code can be executed atomically:

![Figure 6.5](./images/f6-5.png)

![Figure 6.6](./images/f6-6.png)

Imagine this run on multiple threads, whichever thread runs the test_and_set first gets to execute its critical section first. As the first thread will have boolean rv = false (as that is what *lock is init to) then it will set lock to true so that the other processes will have rv = true and they must all wait until the first process finishes its critical section and sets lock=false.

Another method of locking is demonstrated in figure 6.7 * 6.8:

![Figure 6.7](./images/f6-7.png)

![Figure 6.8](./images/f6-8.png)

In this example, the first thread sees the lock == expected (lock == 0/false) meaning there is no lock and it can execute its critical section. It sets the value = new_value meaning it sets lock = 1 and executes its critical section and after it finishes it sets the lock back to zero so the next thread can execute its critical section.

Figure 6.9 uses compare_and_swap() for mutual exclusion but also makes sure that bounded waiting exists by managing each threads waiting status in the waiting[] array:

![Figure 6.9](./images/f6-9.png)

iterates through each process that exists (using j = (i + 1) % n;), Either Turns the lock off in the process that turned it on (if no other threads are waiting), or it sets the first thread (index wise) waiting to false so that it can then run its critical section. (The lock value stays at 1 until we run the critical section for all threads and we get to our last thread that completed the critical section in which the if (j==i) executes and we set lock back to 0).

**Atomic variable**: Atomic operations on basic variables. Meaning if we know that we will be modifying a variable by multiple threads we can declare it as atomic to avoid race conditions. Ex in C:

    _Atomic const int * p1;  // p is a pointer to an atomic const int
    const atomic_int * p2;   // same
    const _Atomic(int) * p3; // same

    void increment(atomic_int *v) {
         int temp;
         do {
             temp = *v;
         } while(temp != (compare_and_swap(v, temp, temp+1));
    }

    increment(&sequence);

Note: atomic variables dont solve all race conditions (like in bounded-buffer problem). atomic vars usually used for counters and sequence generators.

**Mutex-lock**: A software level (high level not hardware based/low level) method for handling mutual exclusion on critical sections of code. A function has to use acquire() to acquire the lock to be able to execute critical sections, then it must release() the lock for other threads to use it on their critical sections. A process tries to acquire a lock by checking if its available, if it isnt then its blocked to wait for it to become available. The entire process is demonstrated in figure 6.10:

![Figure 6.10](./images/f6-10.png)

    acquire() {
        while (!available)
        ; /* busy wait */
        available = false;
    }

    release() { available = true; }

    Each can be performed atomically using compare_and_swap() (CAS).

**Spinlock**: A locking method specifically used for short duration locks for multiprocessor systems. The process "spins" while waiting meaning it uses up the CPU to wait. Short duration means the lock will be held for a duration of less than two context switches (waiting on a lock requires two context switches 1. a context switch to move the thread to the waiting state and 2. a second context switch to restore the waiting thread once the lock becomes available). Example of using spinlock for acquire():

    void acquire() {
        while(test_and_set(&lock))
        ; /* spin */
        /* critical section */
        lock = false;
    }

**Semaphore**: Using an integer value S to manage concurrent threads/processes. Essentially used as a lock flag on an instance of a variable. A positive value means there is no lock and the variable is allowed to be modified, and a value of less than or equal to zero means the value is currently locked from being accessed and modified. There are two types of semephores **binary semaphore** and **counting semaphore**.

    wait(S) {
        while (S <= 0)
        ; // busy wait
        S--;
    }

    signal(S) { S++; }

    wait(S)
    critical section;
    signal(S)



**Binary semaphore**: Sets the S value to be binary, meaning we are managing the access to a single instance of a variable among several threads. Ex:

    S = 1; //initally S is 1 meaning we can access our shared variable (its free and not locked)
    
    while (S <= 0) // since S is 1 we continue through the loop and decrement S-- so that no other threads can proceed and all get stuck on the while loop in wait
    
    We then execute our critical section
    then we run signal(S) and increment S so that its free and another thread can proceed with its critical section

**Counting semaphore**: More than once instance of a variable. Say for example we can have multiple edits of the same value and compare or combine these somehow. Ex: 2 different versions of a variable

    S = 2; // 2 instances of variable (we can have at max 2 threads updating our variable at the same time since we have 2 copies)

    while (S <= 0) 
    S--; // this runs for the first two threads since we have 2-1=1, then 1-1=0 
    critical section //two threads run critical section at once
    signal(S) // can update S to be 0+1 = 1 (then another thread picks it up and decrements it back to zero) or the other thread finishes as well and we have 0+1 = 1 and 1+1= 2.

\*Note: S is a atomic value meaning that (in the low level hardware side) we have code that runs a wait() and release() wrapped around updates and reads of S so that only one thread can update S at once (and we dont have mutliple threads checking that S=1 and both setting S-- == 0).*

One issue with this semaphore implementation of mutex locking is that it uses **busy waiting** which means that it uses up CPU computation when waiting (as it gets stuck in a while loop). We dont want to use this when we expect a thread to wait for a longer than short duration time. To combat this we can add a self suspending/sleeping technique. 

    value = val; // initialized to the number of available resources

    Semaphore::Wait(Thread T) {
        intr_disable();
         value = value - 1;
         if(value < 0) { // |value| is the number of waiting threads
             queue_add(Q, T);
             thread_block(T);
         }
         intr_enable();
    }

    Semaphore::Signal() {
        intr_disable();
        value = value + 1;
         if(value <= 0) // if there is a waiting thread
             thread_unblock(queue_remove(Q));
         intr_enable();
    }

    Here we increment and decrement our S value each time so it represents the amount of threads waiting to use our value in the queue. Absolute value on negative value is the amount waiting.


**Conditional variable**: Abstraction for conditional synchronization. "a queue of threads waiting for a specific event inside the critical section". "the condition of the condition variable is defined based on the data
protected by a mutex lock".

**Semaphore vs Conditional variable**: Semaphore manages multiple threads on multiple instances of a variable using a variable instance counter. While conditional variable is the interaction of multiple threads on handling a single resource/variable based on a set of conditions. "Semaphores ensure there are no more than a set number of threads using resources simultaneously, while condition variables make sure threads interact correctly around specific conditions."

    Conditional variable example:

    void producer() {
        while (1) {
            while (count == BUFFER_SIZE); // Busy wait
            buffer[count++] = 1; // Produce an item
            buffer_not_empty = 1; // Signal that buffer is not empty
            if (count == BUFFER_SIZE) {
                buffer_not_full = 0; // Signal that buffer is full
            }
            // other code
        }
    }
    
    void consumer() {
        while (1) {
            while (!buffer_not_empty); // Busy wait
            int item = buffer[--count]; // Consume an item
            buffer_not_full = 1; // Signal that buffer is not full
            if (count == 0) {
                buffer_not_empty = 0; // Signal that buffer is empty
            }
            // other code
        }
    }

**Monitor**: A class that ties (private) data and methods (including the synchronization operations) together. This is done to create a thread safe data handler. As with Semaphore we still run the risk of the compiler switching around lines and functions (since our entire handler is made up of lines and the raw functions this can create signal before wait call or double wait calls for example). See figure 6.11 for an example structure:

![Figure 6.11](./images/f6-11.png)

A lock flag is defined inside the monitor as well as the flexibility to define 0 or more custom conditions that define how to synchronize our shared data updates:

    condition x, y;
    x.wait();
    x.signal(); // resumes suspended process (if none exist then nothing happens (this is extra benefit that semaphore doesnt provide))
    /* wait() and signal() are the only updating methods for our condition variables */

Handling a waiting thread (T2) when x.signal() is called (on thread T1) boils down to two methods:

1. **Signal and wait (Hoare-style monitors)**: T1 gives execution to T2, T1 gets put in a special signal queue, and waits until T2 terminates or waits again.
2. **Signal and continue (Mesa-style monitors)**: T2 waits (gets put in ready queue) until T1 terminates or waits (another waiting thread could get execution before T2 (no special priority given to T2)).

**Deadlock**: Every process/thread in a set is waiting for a signal from another process/thread. All threads are stuck as they are all waiting for a lock to be released and there are no threads to release the lock (since they are all waiting). Ex:

    P0: wait(T1), P1: wait(T2) 
    Now both P0 and P1 are waiting for each other to signal and release the lock.

**Hoare Monitor implementation with semaphores**:

    public:
        void ConditionWait(); // calls cvar.wait() and implements Hoare semantics
        void ConditionSignal(); // calls cvar.singal() and implements Hoare semantics
        void someMethod(); // user-defined methods working on shared data
        
    private:
        <shared data>; // data being protected by monitor
        semaphore lock; // controls entry to monitor
        semaphore cvar; // suspends a thread on a wait
        int waiters_cvar; // number of threads waiting on a cvar
        semaphore next; // suspends this thread when signalling another
        int waiters_next; // number of threads suspended on next

    Monitor::Monitor(int N) {
        cvar = N; // initialized to N
        lock = 1; // initialized to 1 as nobody is in the monitor
        next = 0; // initialized to 0 as nobody is suspended because of signalling
        waiters_next = 0;
        waiters_cvar = 0;
    }

    void Monitor::someMethod() {
        lock.Wait(); // lock the monitor
        <ops on data and calls to ConditionWait() and ConditionSignal()>
        if (waiters_next > 0)
            next.Signal(); // resume a suspended thread
        else
            lock.Signal(); // allow a new thread into the monitor
    }

    void Monitor::ConditionWait() {
        waiters_cvar += 1;// increment the number of waiters
        if(waiters_next > 0)
            next.Signal(); // resume a suspended thread
        else
            lock.Signal(); // allow a new thread in the monitor
        cvar.wait(); // wait on the condition
        waiters_cvar -= 1;// on waking up decrement the number of waiters
    }
    void Monitor::ConditionSignal() {
        if (waiters_cvar > 0) { // don't signal cvar if nobody is waiting
            waiters_next += 1; // increment the number of suspended threads
            cvar.Signal(); // awaken a waiting thread
            next.Wait(); // wait for it to finish or wait again
            waiters_next -= 1; // decrement the number of suspended threads
        }
    }

# Chapter 7

**Bounded-Buffer problem**: Demonstrates the nececity of semaphore using a common real life functionality:

    count = 0; // to keep track of the used spaces
    last = 0;

    void BoundedBufferMonitor::P(Item item){ //::P -> wait() (check if instance of data is available and if it is then take it (S--) 
        lock.Acquire();
        while(count == N)
            empty.Wait(lock);
        buffer[last] = item;
        last = (last + 1)%N;
        count++;
        full.Signal();
        lock.Release();
    }
    void BoundedBufferMonitor::C(Item &item){ // ::C -> signal() (done with working on data so now free it (S++) 
        lock.Acquire();
        while(count == 0)
            full.Wait(lock);
        item = buffer[(last-count)%N]; // first=(last-count) mod N
        count--;
        empty.Signal();
        lock.Release();
    } 

    ::P -> adding data to the buffer. 
        place lock on buffer so no other threads can take manipulate its data
        Wait until buffer is not full
        add data to end of buffer
        indicate that is full
        release the lock on the buffer
        
    ::C -> taking data from the buffer
        place lock on buffer so no other threads can take manipulate its data
        Wait until buffer is not empty
        read data from end of buffer
        indicate that is empty
        release the lock on the buffer
    
**Readers-Writers problem**: Common problem used for testing synchronization methods. Threads seperated into readers and writers where readers can only read the data (and can do so concurrently with no issue). While writers have to have exclusive access to data (using locks) to make sure no mixups of data happen. Ex code:

    semaphore rw_mutex = 1;  // lock semaphore for reading and writing
    semaphore mutex = 1;  // lock semaphore for updating the read_count
    int read_count = 0;  // how many readers we have activly reading the data

    // writer process
    while (true) { 
        wait(rw_mutex);
        ...
        /* writing is performed */
        ... 
        signal(rw_mutex);
    }
    // ^ wait for a reader/writer resource to be avaibale, then write, then release the reader writer lock

    // reader process
    while (true) { 
        wait(mutex);
        read count++;
        if (read count == 1)
            wait(rw_mutex); 
        signal(mutex);
        ...
        /* reading is performed */
        ... 
        wait(mutex);
        read count--;
        if (read count == 0)
            signal(rw mutex); 
        signal(mutex);
    }
    // wait for resource to update the reader count, 
    // if this is the first reader then wait for a lock so that we can stop writers from writing while we are trying to read (now can add as many readers as we want as we have writing lock the entire time)
    // release the update reader counter lock

    // perform our reading

    // wait for a reader count lock to be available to use
    // remove our reader
    // if we just removed our last reader then release the reader writer lock so that we can write to our data again
    // release the reader count lock

**Dining-philosophers problem**: Common problem to demonstrate synchronization among a large class of concurrency-control problems. (Many things are trying to take control concurrently). Each philosopher is either thinking or eating rice and they need two chopsticks to do so. In a table of 5 philosophers, each two philosophers share one chopstick. See figure 7.5:

![Figure 7.5](./images/f7-5.png)

We can implement a solution where each philosopher has to wait for each of their required chopsticks to be free:

    while (true) { 
        wait(chopstick[i]); 
        wait(chopstick[(i+1) % 5]);
        ...
        /* eat for a while */
        ... 
        signal(chopstick[i]); 
        signal(chopstick[(i+1) % 5]);
        ...
        /* think for awhile */
        ...    }
This however causes an issue where in some cases we can have all 5 threads try to grab their left chopstick, then each philosopher is stuck in a wait for a chopstock (this is a deadlock). 

One solution to avoid deadlock is to make sure both chopsticks are available before a philosopher picks one up to eat. See figure 7.7 for the code solution:

![Figure 7.7](./images/f7-7.png)

Eating code for the functions defined above:

    DiningPhilosophers.pickup(i);
    ... eat ...
    DiningPhilosophers.putdown(i);


Philosopher i can set the variable state[i] = EATING only if her two neighbors are not eating. 

    If one is eating then we set the self[i].wait()

One issue with this code is that it does not handle starvation. In order to guarantee that starvation doesnt happen we need to guarantee that threads will be unblocked in the same order that they are blocked (FIFO). So we can implement a waiting queue that adds newly blocked threads to the tail and unlocks threads from the head of the queue.

# Chapter 19

**Distributed System**: A collection of computer programs (or processors) known as nodes that each have their own memory and clock (or their own OS) and communicate with each other over a common network and work together to acheive some goal. From the point of view of a specific node, its resources are local whereas all other nodes and resources are remote. One example of a distributed system is the internet.

Below in figure 19.1 is an example of a client-server distributed system:

![Figure 19.1](./images/f19-1.png)

**Resource sharing**: "Provides mechanisms for sharing files at remote sites, processing information in a distributed database, printing files at remote sites, using remote specialized hardware devices such as a supercomputer or a GPU."

**Computation speedup**: Being able to split the computations of a task among many nodes in a distributed system to speed up the computation speed.

**Load balancing**: Taking some of the computation tasks from a node that is overloaded with requests and sending them to a more lighlty loaded node.

A distributed system is reliable in the fact that if one node fails, it doesnt affect the other nodes. Unless nodes are assigned for a specific task that other nodes are dependent upon (like a global database for example). In that case backup nodes for specifc task nodes are expected to be made.

Distributed systems can be connected by two types of networks: local-area networks (LAN) and wide-area networks (WAN).

**Local-area network (LAN)**: Nodes are distributed over a small geographical area like inside a single building or a number of adjacent buildings. One of the main advantages of LAN over WAN is that due to each node/computers short distance the connection/communications links usually have a higher speed and lower error rate. Connection links are usually either WIFI or an ethernet cable.

Figure 19.2 demonstrates an example of a LAN inside a home/office:

![Figure 19.2](./images/f19-2.png)

**Wide-area network (WAN)**: Nodes (or systems) distributed over a large area like the United States. Connection links are usually telephone lines, leased (dedicated data) lines, optical cable, microwave links, radio waves, and satellite channels. The communication links are controlled by **routers** that are responsible for directing traffic to other routers and networks but also the general information transfer through the links to the nodes. Figure 19.3 below demonstrates a distributed system with network hosts (nodes) on the outside and routers on the inside:

![Figure 19.3](./images/f19-3.png)


In this figure we can imagine how the internet is structured. The network hosts (or nodes) are computers connected to LAN's. The LAN's are connected to the internet via regional networks, and the regional networks are interlinked with routers to form the worldwide network (the internet).

Even though WAN's are generally slower than LAN's, WAN's connections that link major cities may have very fast transfer rates through fiber optic cables.

**Transmission Control Protocol/Internet Protocol (TCP/IP)**: A communication standard link that is used by the internet for its distributed system. It transmits data (back and forth) between a client and a server by first establishing a connection and then using that connection to share data by breaking it down into smaller packets and piecing them back together on the other side. It can be broken down into 4 layers:

1. Application layer - Interactive layer for the user, handles data exchange on the application. An example is http on a website
2. Transport layer - Handles the data preperation, splits the data into TCP/UDP packets and places them inside a packet order and controlling flow. (Kinda like the packaging center for a mailroom)
3. Internet layer - Encodes and decodes routing IP values onto the data packets. (Kinda like the mailroom person adding mailing stamps and reading mailing stamps)
4. Link layer - Handles the transmission of data over a physical medium (handling error detection and data recovery). (Kinda like the mailtruck driving the mail).

TCP is a reliable connection-oriented transport protocol (compared to UDP). TCP implements a byte stream that allows for data to be sent in order and uninterrupted. It does so with the following steps:

1. Establishes a **acknowledgment packet (ACK)** which is a required message that the receiver has to send to the sender to acknowledge that they received the data from the sender.
2. Utilizes **sequence numbers** which act as unique id's and increase incrementally with each data packet so the sender can make sure its receiving the data in order.
3. Uses control packets to initialize a connection and to tear down a connection. Known as a **three-way handshake** as the following 3 signals are exchanged: SYN, SYN+ACK, ACK.

Figure 19.11 demonstrates an example of a TCP data transfer. The example can be broken down into the following steps:

1. After a connection is established, the client sends a request packet to the server with the sequence number 904.
2. The server sends back an ACK for the data 904 request.
3. The server then sends its own data to the client identifiable with the sequence number 126.
4. The client receives data 126 and sends back an ACK for it.
5. The server then tries to send data with sequence number 127 to the client, but the data does not reach and gets lost.
6. The server reached its time limit waiting for an ACK for data 127 and so it reaches a timeout and resends data 127.
7. This time the client receives data 127 and sends an ACK for it to the server.
8. The server sends the next piece of data to the client (with sequence number 128).
9. The client receives the data and sends back an ACK for it but the ACK is now lost and never delivers.
10. The server reaches its timeout for the ACK for data 128 and resends data 128.
11. The client receives data 128 again, marks it as a duplicate and resends the ACK for it (knowing that its previous ACK must have not been delivered).

![Figure 19.11](./images/f19-11.png)

**Flow control**: Mechanism utilized by TCP to regulate the flow of data packets. Prevents the sender from overrunning the capacity of the receiver. It does this by encoding a message in ACK's that tell the sender to either slow down their data sending or to speed it up (and this is determined by many factors including the receivers hardware and the current speed of the network).

**Congestion control**: Determines the slow down or speed up message in ACK from flow control by analyzing how many data packets are being lost/dropped (as router drops data packets if overwhelmed by requests).

**Access control (MAC) address**: The address attached to a datapacket that is being sent within a local network (LAN). Every Ethernet/WiFi device has a unique medium access control (MAC) address. 

**Address resolution protocol (ARP)**: The address generated for a data packet that is being sent from a system on a local network to another system on a different local network. Also known as: IP to MAC address mapping. In this mapping of ARP to IP we can then send packets of data that have a certain ARP which is mapped to certain MAC addresses so we can send data to only a certain amount of specific local networks.

**User Datagram Protocol (UDP)**: A bare bones quick and cheap version of TCP. The UDP header only contains four fields: source port number, destination port number, length, and checksum. Packets of data sent quickly but with no guardrails meaning we can lose some data packets in the process (or get them out of order) and its up to the application to handle this by either requesting the missing packet again or deciding to make do with whatever packets it did receive. Also known as a **connectionless** protocol as there is no connection setup to establish a communication state between the servers nor is there a connection teardown, UDP simply sends the data right away and hopes that the recipient is able to receive it.